In [31]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import sys
import os
sys.path.insert(0, "/var/home/d.svirchkov/custom_scripts/scripts/")

from create_features_pyspark import *

In [33]:
hdfs_path = "hdfs dfs" #"/usr/share/hadoop/bin/hdfs dfs"

In [34]:
full_path = '...'

---

In [20]:
train = spark.read.parquet(full_path + 'train')
train.registerTempTable('train')

In [21]:
train.limit(10).toPandas()

,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,__index_level_0__
0,823300,0.297589,1,4,2,4,2,1,3,2,2,1,93,2,2,10,240,48,-1,1,4280508
1,823300,0.313568,1,1,2,1,1,1,1,1,1,1,93,1,2,10,240,48,0,2,4280509
2,823300,0.305395,1,1,2,1,1,1,1,1,4,1,2,3,6,15,238,46,53,3,4280510
3,823300,0.302912,1,1,2,1,1,1,1,1,1,1,93,1,4,15,236,46,48,4,4280511
4,823300,0.173450,1,1,2,1,1,1,1,1,10,1,93,7,3,15,235,46,24,5,4280512
5,823300,0.278876,1,1,2,1,1,1,1,1,1,1,93,1,3,15,235,46,0,6,4280513
6,823300,0.426544,1,4,2,4,2,1,3,2,2,1,93,2,2,9,233,46,42,7,4280514
7,823300,0.285830,1,1,2,1,1,1,1,1,1,1,93,1,2,10,226,20,168,8,4280515
8,823300,0.354985,1,1,2,1,1,1,1,1,1,1,93,1,2,10,226,20,0,9,4280516
9,823300,0.296745,1,1,2,1,1,1,1,1,1,1,93,1,6,15,224,36,53,10,4280517


In [196]:
%%time
create_train_parquet_v1(spark, full_path, hdfs_path, max_history_days=365)

train_transactions_data4features saved


In [ ]:
%%time
create_all_features_selected(
    spark,
    full_path,
    hdfs_path,
    use_keys=[],
    feats_to_create=[], 
    lim=200000, 
    time_periods_in_days=[365], 
    bd_list=['train_transactions'],
    source_1_list=['cc', 'dc', 'ac'],      # operation_type_group
    source_2_list=['outflow', 'inflow'],   # income_flag
    source_3_list=['ecom1', 'ecom2'],      # 
    categories_groups_dict='default',
    )

# ~45 minutes

---

`ноутбук с отбором фич и тюнингом гиперпараметров получился слишком увестистым, поэтому решили убрать, так же как и ноутбук с  Exploratora Data Analysis`

In [29]:
selected_features = ['dc_inflow_ecom1_365days_op_type1_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat7_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat2_mean_amnt',
 'ac_inflow_ecom1_365days_op_type5_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat4_mean_hourdiff',
 'dc_inflow_ecom1_365days_op_type1_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat3_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat4_cnt',
 'ac_inflow_ecom1_365days_mcc_cat7_total_amnt',
 'dc_inflow_ecom1_365days_op_type2_mean_amnt',
 'dc_inflow_ecom1_365days_op_type3_total_amnt',
 'dc_inflow_ecom1_365days_mcc_cat2_first_act',
 'dc_inflow_ecom1_365days_mcc_cat5_mean_hourdiff',
 'cc_outflow_ecom1_365days_op_type4_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat6_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat5_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat9_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat13_mean_hourdiff',
 'dc_inflow_ecom1_365days_op_type5_mean_amnt',
 'dc_inflow_ecom1_365days_op_type5_cnt',
 'dc_inflow_ecom1_365days_mcc_cat4_mean_amnt',
 'ac_inflow_ecom1_365days_mcc_cat2_total_amnt',
 'ac_inflow_ecom1_365days_op_type7_mean_hourdiff',
 'ac_inflow_ecom1_365days_mcc_cat4_total_amnt',
 'dc_inflow_ecom1_365days_mcc_cat1_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat1_last_act',
 'dc_inflow_ecom2_365days_op_type2_mean_hourdiff',
 'cc_outflow_ecom1_365days_op_type4_first_act',
 'dc_inflow_ecom1_365days_op_type2_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat7_mean_amnt'
]

In [30]:
# dc - debit card, cc - credit card, ac - all cards
# outflow / inflow - списания/пополнения
# ecom1 / ecom2 - флаг ecommerce operation 

# mcc_cat 1...28 - merchant category code
# op_type - operation_type 1 ... 22

---

In [ ]:
%%time
create_test_parquet(spark, full_path, hdfs_path, max_history_days=365)

In [26]:
%%time
create_all_features_selected(
    spark,
    full_path,
    hdfs_path,
    use_keys=[],
    feats_to_create=selected_features, 
    lim=200000, 
    time_periods_in_days=[365], 
    bd_list=['test_transactions'],
    source_1_list=['cc', 'dc', 'ac'],      # operation_type_group
    source_2_list=['outflow', 'inflow'],   # income_flag
    source_3_list=['ecom1', 'ecom2'],      # 
    categories_groups_dict='default',
    )

--------------------
joining: mcc_cat1
Done stats_mcc_cat1 // nfeats: 2 // time 27.4
--------------------
joining: mcc_cat2
Done stats_mcc_cat2 // nfeats: 3 // time 24.9
--------------------
joining: mcc_cat3
Done stats_mcc_cat3 // nfeats: 1 // time 15.7
--------------------
joining: mcc_cat4
Done stats_mcc_cat4 // nfeats: 4 // time 22.9
--------------------
joining: mcc_cat5
Done stats_mcc_cat5 // nfeats: 2 // time 16.0
--------------------
joining: mcc_cat6
Done stats_mcc_cat6 // nfeats: 1 // time 14.1
--------------------
joining: mcc_cat7
Done stats_mcc_cat7 // nfeats: 3 // time 22.7
--------------------
joining: mcc_cat8
Skip stats_mcc_cat8 // nfeats: 0
--------------------
joining: mcc_cat9
Done stats_mcc_cat9 // nfeats: 1 // time 12.9
--------------------
joining: mcc_cat10
Skip stats_mcc_cat10 // nfeats: 0
--------------------
joining: mcc_cat11
Skip stats_mcc_cat11 // nfeats: 0
--------------------
joining: mcc_cat12
Skip stats_mcc_cat12 // nfeats: 0
--------------------
joini